In [1]:
import fcn32_vgg 
import numpy as np
import tensorflow as tf
import skimage.io
import skimage.transform
import utils
import scipy as scp
import scipy.misc
import loss
import pandas as pd
from tensorflow.python.framework import ops
from skimage.color.adapt_rgb import adapt_rgb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import uns

In [3]:
sess=tf.InteractiveSession()

In [4]:
labels= tf.placeholder(tf.int32,[None,420,580,2])
batch_images = tf.placeholder(tf.float32,[None,420,580,3])

In [5]:
### put your path to the pre-trained weights here...
vgg_path = '../tensorflow-fcn/vgg16.npy'

In [6]:
#net = fcn32_vgg.FCN32VGG(vgg16_npy_path=vgg_path)
#with tf.name_scope("content_vgg"):
#        net.build(batch_images, num_classes=2,debug=False)


In [7]:
net = fcn32_vgg.FCN32VGG(vgg16_npy_path=vgg_path)
net.build(batch_images, num_classes=2,debug=False)

print('Finished building Network.')

npy file loaded
Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
Layer name: fc6
Layer shape: [7, 7, 512, 4096]
Layer name: fc7
Layer shape: [1, 1, 4096, 4096]
Layer name: fc8
Layer shape: [1, 1, 4096, 1000]
Finished building Network.


In [8]:
output = tf.cast(net.upscore,tf.float32)

In [9]:
cost_function = loss.loss(output,labels,2)

In [10]:
train_step = tf.train.AdamOptimizer(1e-6).minimize(cost_function)

In [11]:
sess.run(tf.initialize_all_variables())

In [12]:
### the training data

In [12]:
from uns import training

In [13]:
my_batch  = uns.batch(training.iloc[range(5)])

In [14]:
my_batch_arr= my_batch.array_rgb()

In [15]:
my_batch_masks = my_batch.array_masks()

In [16]:
my_batch_masks.shape

(5, 420, 580, 2)

In [16]:
## test 1:
## making a prediction using pretrained weights 

In [25]:
#im1=my_batch_arr[0]

In [26]:
#pixelsOut = sess.run(prediction,feed_dict={batch_images:np.array([im1])})

In [30]:
#pixelsOut.sum()

54152.0

In [26]:
#out=sess.run(net.upscore,feed_dict={batch_images:np.array([im1])})

In [34]:
#out.shape

(1, 420, 580, 2)

In [23]:
## test cost function

In [17]:
cost=sess.run(cost_function,feed_dict={batch_images:my_batch_arr,labels:my_batch_masks})

In [18]:
cost

1.0063345

In [19]:
### test one step of training loop

In [20]:
train_step.run(feed_dict={batch_images:my_batch_arr,labels:my_batch_masks})

In [21]:
new_cost=sess.run(cost_function,feed_dict={batch_images:my_batch_arr,labels:my_batch_masks})

In [22]:
### it went down! 

In [22]:
new_cost

0.95543444

In [18]:
def build_batch():
    pass

In [42]:
def train_network(num_batches,update_period=50,metric=None):    
    for i in range(num_batches):
        batch= build_batch()
        if i%update_period ==0:
            print( "Batch " +str(i))
        #train_error = metric.eval(feed_dict={
        #batch_images:batch[0], y_: batch[1]})
        train_step.run(feed_dict={batch_images:batch[0],labels:batch[1]})